In [50]:
from random import shuffle
import glob,h5py, cv2
import numpy as np
from numpy import dstack
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
from scipy.signal import resample
width = 128
height = 128
channels = 3
duration = 24.2
fps = 25
sub = 'Blake2'
num_frames = duration*fps # axis = 1 dimension, also the # of frames are going to be padded to this value.
filename = 'Datasets/'+sub+'_margins_and_ctc_label_no_nucleus.hdf5'
print('File to be created: ' + filename)

File to be created: Datasets/Blake2_margins_and_ctc_label_no_nucleus.hdf5


In [51]:
# load labels
trainy = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/microDoppler/*.txt'
trainy_files = sorted(glob.glob(trainy))
print('Num. of label files: '+str(len(trainy_files)))
for i in range(0,5):
    print(str(i+1)+'. ' +trainy_files[i])

Num. of label files: 23
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/microDoppler/11010003_1617741768_1.txt
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/microDoppler/11010003_1617741768_2.txt
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/microDoppler/11010003_1617741768_3.txt
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/microDoppler/11010003_1617741768_4.txt
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/microDoppler/11010003_1617741768_5.txt


In [52]:
# load margins
trainpath = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/*/envelopes/*.txt'
train_files2 = sorted(glob.glob(trainpath))
len(train_files2)
train_files2[0]
train_files = []
for i in range(0,len(trainy_files)):
#     fname = trainy_files[i][-23:].replace("txt","") + "txt"
#     fname = trainy_files[i][-30:].replace("txt","txt") # -23 for nonbinary labels
#     fname = fname.replace('_binary','')
#     fname = trainy_files[i]
    slash_idx = trainy_files[i].rfind('/')
    fname = trainy_files[i][slash_idx:]
    idx = [j for j, s in enumerate(train_files2) if fname in s] #train_files2.index(fname) 
    if len(idx) == 1:
        train_files.append(train_files2[idx[0]])
    else:
        print('2 identical filenames at i=',str(i))
print('Num. of label files: '+str(len(train_files)))
for i in range(0,5):    
    print(str(i+1)+'. '+train_files[i])

Num. of label files: 23
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/envelopes/11010003_1617741768_1.txt
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/envelopes/11010003_1617741768_2.txt
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/envelopes/11010003_1617741768_3.txt
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/envelopes/11010003_1617741768_4.txt
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/envelopes/11010003_1617741768_5.txt


In [53]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

## load margins

In [54]:
margins = []

cnt = 1
for file in train_files:
    clear_output(wait=True)
    print('Loading margins #'+str(cnt))
    yx = load_file(file)
    margins.append(np.squeeze(np.array(yx)))
    cnt += 1
# margins = np.squeeze(np.array(margins))
margins = np.array(margins)

print(margins.shape)

Loading margins #23
(23,)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


In [55]:
for i in range(18):
    print(margins[i].shape)

(1489,)
(1489,)
(1489,)
(1489,)
(1489,)
(1489,)
(1489,)
(1489,)
(1489,)
(1489,)
(1489,)
(1489,)
(1489,)
(1489,)
(1488,)
(1488,)
(1488,)
(1488,)


In [56]:
# with nucleus and retraction
seq1 = np.array([1,2,19,4,20,19,5,20,19,6,20,3])
seq2 = np.array([1,2,19,7,20,19,8,20,19,9,20,3])
seq3 = np.array([1,2,19,10,20,19,11,20,19,12,20,3])
seq4 = np.array([1,2,19,13,20,19,14,20,19,15,20,3])
seq5 = np.array([1,2,19,16,20,19,17,20,19,18,20,3])
sequences = [seq1,seq2,seq3,seq4,seq5]
chr(ord('@')+19)
int(trainy_files[0][-22])

1

In [57]:
# without nucleus and retraction
seq1 = np.array([1,2,4,5,6,3])
seq2 = np.array([1,2,7,8,9,3])
seq3 = np.array([1,2,10,11,12,3])
seq4 = np.array([1,2,13,14,15,3])
seq5 = np.array([1,2,16,17,18,3])
sequences = [seq1,seq2,seq3,seq4,seq5]
chr(ord('@')+19)
# int(trainy_files[51][-22])

'S'

In [58]:
sequences[0]

array([1, 2, 4, 5, 6, 3])

In [59]:
# labels = []
# label = ''
# for ch in sequences[0]:
#     label = label + chr(ord('@')+ch)
# labels.append(label)
# label

In [60]:
labels = []

cnt = 1
for file in trainy_files:
    clear_output(wait=True)
    print('Loading labels #'+str(cnt))
    seqnum = int(file[-22])-1
    label = ''
    for ch in sequences[seqnum]:
        label = label + chr(ord('@')+ch)
    labels.append(label)
    cnt += 1
labels = np.array(labels)  
print(labels.shape)

Loading labels #23
(23,)


In [ ]:
labels[21]

In [43]:
# Blake's
from itertools import groupby 
labels = []

cnt = 1
for file in trainy_files:
    clear_output(wait=True)
    print('Loading labels #'+str(cnt))
    yx = load_file(file)
    y = [i[0] for i in groupby(yx)]
    label = ''
    for ch in y[0]:
        if ch != 0:
            label = label + chr(ord('@')+ch)
    label = [i[0] for i in groupby(label)]
    label = ''.join(label)
    labels.append(label)
    cnt += 1
labels = np.array(labels)  
print(labels.shape)

Loading labels #18
(18,)


In [61]:
label

'ABPQRC'

In [62]:
# # train
# loaded_tr = list()
# winsize = 0.2 # sec
# totalwin = int(30/winsize)
# for name in train_files:
#     data = load_file(name)
#     loaded_tr.append(data)
# # loaded_tr = dstack(loaded_tr) 
# envelopes = np.swapaxes(np.array(loaded_tr), 1, 2)

# loaded_tr = list()
# for name in trainy_files:
#     data = load_file(name)
#     loaded_tr.append(data)
# labels = np.array(loaded_tr)    
# labels = to_categorical(np.swapaxes(labels, 1, 2))

# print(envelopes.shape)
# print(labels.shape)

#### Train-Test Split

In [63]:
x_train, x_test, y_train, y_test = train_test_split(margins, labels, test_size=0.2, random_state=1)

# if Reshape == 1:
#     x_train = x_train.reshape(-1,fps,x_train.shape[2],x_train.shape[3],x_train.shape[4])
#     x_test = x_test.reshape(-1,fps,x_test.shape[2],x_test.shape[3],x_test.shape[4])
#     y_train = y_train.reshape(-1,fps,y_train.shape[2])
#     y_test = y_test.reshape(-1,fps,y_test.shape[2])
    
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print('# of frames in a class:')
# Counter(np.concatenate(np.argmax(y_train, axis=2),axis=0))

(18,)
(18,)
(5,)
(5,)
# of frames in a class:


In [64]:
# UTF encoding to store strings into hdf5
utftrain = [y.encode('utf8') for y in y_train]
utftest = [y.encode('utf8') for y in y_test]

In [65]:
x_train[0].shape

(1488,)

In [66]:
f = h5py.File(filename, mode='w')
# f.create_dataset("train_img", x_train.shape, np.uint8)
# f.create_dataset("test_img", x_test.shape, np.uint8) 
dt = h5py.special_dtype(vlen=str)
f.create_dataset("train_labels", (len(utftrain),1), dtype = 'S12',data=utftrain)
f.create_dataset("test_labels", (len(utftest),1), dtype = 'S12',data=utftest)

# blake's
dt = h5py.special_dtype(vlen=np.dtype('float64'))
f.create_dataset('train_img', (x_train.shape[0],), dtype=dt)
f['train_img'][...] = x_train
f.create_dataset('test_img', (x_test.shape[0],), dtype=dt)
f['test_img'][...] = x_test

# f["train_img"][...] = x_train
# f["test_img"][...] = x_test
# f["train_labels"][...] = utftrain
# f["test_labels"][...] = y_test
f.close()
print(filename+' created.')

Datasets/Blake2_margins_and_ctc_label_no_nucleus.hdf5 created.


In [64]:
f.close()